In [ ]:
# Initialize Otter
import otter
grader = otter.Notebook()

# CS187
## Lab 2-1 – Sequence labeling with hidden Markov models

In [1]:
from collections import defaultdict

Hidden Markov models (HMM) are a fundamental generative method for sequence labeling NLP tasks such as part-of-speech tagging (as in the present lab) and information extraction (as in the second project segment). In this lab, you'll train, apply, and evaluate some simple sequnce labeling algorithms culminating in HMM.

To keep things manageable, the dataset you'll use will involve very few word types, only six (plus a special beginning of sentence token), but these are quite ambiguous with regard to part of speech. We'll use the following codes for parts of speech:

In [2]:
parts_of_speech = [
    "<bos>", # beginning of sentence marker
    "N",     # noun
    "V",     # main verb
    "M",     # modal verb
    "P",     # preposition
    "A",     # adjective
    "R"      # adverb
]

The vocabulary of word types, along with their parts of speech, is given by the following dictionary:

In [3]:
vocabulary = {
    "<bos>":   ["<bos>"],
    "can":     ["N", "V", "M"],
    "canned":  ["A", "V"],
    "canners": ["N"],
    "fish":    ["N", "V"],
    "for":     ["P"],
    "not":     ["R"]
}

Here are a few sentences constructed with these words:

In [4]:
text = """
    <bos> canners canned fish
    <bos> can canners can fish
    <bos> fish can not fish
    <bos> can fish can fish can
    <bos> canners fish fish for can
    <bos> canners can fish for fish
    <bos> canners fish for fish
    <bos> fish can canned fish
    <bos> canners can not can canned fish
    <bos> fish can can fish for canners
"""

and the corresponding POS sequences, for the first few sentences. Complete the rest.

In [5]:
#TODO
text_pos = """
    <bos> N V N
    <bos> M N V N
    <bos> N M R V
    <bos> M N V N N
    <bos> N V N P N
    <bos> N M V P N

    ...fill this out for the remaining four sentences...
        
"""

In [6]:
#Solution
text_pos = """
    <bos> N V N
    <bos> M N V N
    <bos> N M R V
    <bos> M N V N N
    <bos> N V N P N
    <bos> N M V P N
    <bos> N V P N
    <bos> N V A N
    <bos> N M R V A N
    <bos> N M V N P N
"""

For reference, here is the frequency distribution for each word type and each part of speech it can be used as.

In [7]:
def deconstruct(text):
    result = []
    for line in text.strip().split("\n"):
        result.append([item for item in line.strip().split()])
    return result

tagged_text = [list(zip(sentence, poses))
                 for sentence, poses 
                   in zip(deconstruct(text), deconstruct(text_pos))]
               
counts = defaultdict(lambda: defaultdict(int))
for sentence in tagged_text:
    for type, pos in sentence:
        counts[type][pos] += 1

for type, type_counts in counts.items():
    for pos, count in type_counts.items():
        print(f'{type:7} {pos:5} {count:2}')

## Majority label

$$
   \newcommand{\argmax}[1]{\underset{#1}{\operatorname{argmax}}}
   \newcommand{\Prob}{{\Pr}}
   \newcommand{\given}{\,|\,}
   \newcommand{\vect}[1]{\mathbf{#1}}
   \newcommand{\cnt}[1]{\sharp(#1)}
$$
The first sequence labeling method we'll use is simply to choose for each word the POS label it most frequently occurs as in the training data. The table above provides the required information directly.

Choosing the majority label for a word sequence $\vect{x} = \langle{x_1, x_2, \ldots, x_m}\rangle$ is tantamount to maximizing the probability of the label sequence assuming independence of the label conditioned on the word, that is, selecting the tag sequence $\vect{t} = \langle{t_1, t_2, \ldots, t_m}\rangle$ given by
$$ \argmax{\vect{t}} \prod_{i=1}^m \Prob(t_i \given x_i) $$

How would the majority label method label the following test sentence (which we've marked with the words' correct parts of speech)?

> canners[N] can[V] canned[A] fish[N]

Give your answer in the form of a sequence of POS labels.

In [8]:
#TODO
example_majority_label = ["your", "answer", "here"]

In [9]:
#Solution
example_majority_label = ["<bos", "N", "M", "V", "N"]

By inspection, what is the accuracy of the majority labeling, given as a proportion of the words?

In [10]:
#TODO
example_maj_label_accuracy = "your answer here"

In [11]:
#Solution
example_maj_label_accuracy = 3/5

## Majority bigram label

It may occur to you that what part of speech a word has depends on the context. Suppose we relax the assumption that tag probabilities depend only on the word being tagged, and condition them on the previous word as well. (For the first word in the sentence, we'll condition on that fact. Equivalently, we'll assume that the word "prior" to the first word is a special start token.) In summary, we'll condition on the bigram that ends at the word being tagged:
$$ \argmax{\vect{t}} \prod_{i=1}^m \Prob(t_i \given x_{i-1} x_i) $$

What tag sequence is obtained by using the majority bigram labels for the same sentence?

In [12]:
#TODO
example_majority_bigram_label = ["your", "answer", "here"]

In [13]:
#Solution
example_majority_bigram_label = ["<bos>", "N", "M", "A", "N"]

By inspection, what is the accuracy of the majority bigram labeling, given as a proportion of the words?

In [14]:
#TODO
example_maj_bigram_label_accuracy = "your answer here"

In [15]:
#Solution
example_maj_bigram_label_accuracy = 4/5

## Hidden Markov models

Now we get to the real point, using an HMM model. Recall that in an HMM model, we assume that the joint tag/word sequence is generated by 

1. Selecting a tag sequence according to a Markov model whose states correspond to tags and whose transitions from state $t_i$ to $t_j$ are governed by a _transition probability_ $a_{ij} = \Prob(t_i \rightarrow t_j)$, and then
2. Selecting a word sequence from the tag sequence where for tag $t_i$ we observe word $x_i$ of type $w_j$ governed by an _emission probability_ $b_{i}(w_j) = \Prob(t_i \rightarrow w_j)$.

### Estimating the transition and emission probabilities

We estimate these transition and emission probabilities by looking at the empirical probabilities in the training data, counting and perhaps smoothing as usual. That is, for the (unsmoothed) transition probabilities, we estimate
$$ a_{ij} \approx \frac{\cnt{t_i \rightarrow t_j}}{\cnt{t_i}} $$
and for the emission probabilities
$$ b_i(w_j) \approx \frac{\cnt{t_i \rightarrow w_j}}{\cnt{t_i}} $$

For instance, we note that there are 4 times in the training data where the tag $N$ is followed by the tag $M$, out of the 21 occurrences of the tag $N$. Thus, we estimate the corresponding transition probability $a_{NM} \approx 4/21$.


Similarly, the emission probability $b_M(can)$ for tag $M$ generating the word $can$ is $6/6 = 1$, since every occurrence of the tag $M$ corresponds to the word $can$ in the training data.

Full tables for the transition and emission probabilities are provided below.

In [16]:
# Generate counts
bigram_tag_counts = defaultdict(lambda: defaultdict(int))
unigram_tag_counts = defaultdict(int)
tag_word_counts = defaultdict(lambda: defaultdict(int))
tag_counts = defaultdict(int)

for sentence in tagged_text:
    for (w1, t1), (w2, t2) in list(zip(sentence, sentence[1:])):
        bigram_tag_counts[t1][t2] += 1
        unigram_tag_counts[t1] += 1
    for w, t in sentence:
        tag_word_counts[t][w] += 1
        tag_counts[t] += 1

# Generate transition and emission probabilities
a = defaultdict(lambda: defaultdict(int))
b = defaultdict(lambda: defaultdict(int))

for t1 in parts_of_speech:
    for t2 in parts_of_speech:
        a[t1][t2] = bigram_tag_counts[t1][t2] / unigram_tag_counts[t1]
    for w1 in vocabulary.keys():
        b[t1][w1] = tag_word_counts[t1][w1] / tag_counts[t1]

# Print tables of probabilities

print("Transition probabilities: a_ij")
print(f"{' ':6}", end="")
for t in parts_of_speech:
    print(f"{t:>6}", end="")
print()
for t1 in parts_of_speech:
    print(f"{t1:<6}", end="")
    for t2 in parts_of_speech:
        print(f"{a[t1][t2]:>6.2f}", end="")
    print("")
 
print("\nEmission probabilities: b_i(w_j)")
print(f"{' ':6}", end="")
for w in vocabulary.keys():
    print(f"{w:>8}", end="")
print()
for t in parts_of_speech:
    print(f"{t:<6}", end="")
    for w in vocabulary.keys():
        print(f"{b[t][w]:>8.2f}", end="")
    print()

### An example HMM trellis

<img src="https://github.com/nlp-course/data/raw/master/Resources/hmm-figure.png" width="75%" align=right />

Now consider the HMM generating the example sentence "canners can canned fish". The figure at right contains the _trellis_ for the sentence. The horizontal axis corresponds to the words in the sentence, one at a time. The vertical axis corresponds to the states of the HMM (that is, the parts of speech). The gray arrows that connect a tag on the left to a tag on the right correspond to the transition probabilities. The red arrows that connect a tag to a word directly below correspond to the emission probabilities.

We've highlighted two paths through the trellis from the beginning to the end of the sentence, corresponding to different taggings of the sentence. You should download a copy for ease in doing the next few exercises.

1. Mark on the figure the path corresponding to the majority bigram tagging.
2. Mark on the figure next to the highlighted paths the associated probabilities. The gray arrows should be marked with a transition probability and the red arrows with an emission probability. The tables above should come in handy.
3. Calculate the probability of the majority bigram tagging by multiplying together all of the probabilities along the path that you marked in step 1. Don't forget the emission probabilities.
4. Calculate the probabilities for the two paths that we've marked in the figure. These turn out to be the two paths with the highest probability. 
5. Which tagging has the highest probability according to this HMM?

You'll want to submit a digital copy of your marked-up figure, along with answering the questions below.

In [17]:
#TODO - From 3: what is the HMM probability of the majority bigram tagging?
step_3_probability = "your answer here"

In [18]:
#Solution - From 3: what is the HMM probability of the majority bigram tagging?
step_3_probability = 0.0
# The M -> A transition has probability 0, so regardless of what other 
# probabilities get multiplied in, the probability of the whole path 
# through the trellis is 0.

In [19]:
#TODO - From 4: what is the HMM probability for the two highlighted paths?
step_4_probabilities = ["probability of lower probability path",
                        "probability of higher probability path"]

In [20]:
#Solution - From 4: what is the HMM probability for the two highlighted paths?
step_4_probabilities = [1.00 * 0.80 * 0.32 * 0.31 * 1.00 * 0.33 * 0.10 * 0.55 * 0.59, # <bos> N M V N
                        1.00 * 0.80 * 0.32 * 0.46 * 0.50 * 0.22 * 1.00 * 1.00 * 0.59  # <bos> N V A N
                       ]
step_4_probabilities

In [21]:
#TODO - From 5: which tagging has the highest probability? Give your answer
#        as a string as in the example below.
step_5_highest = "<bos> R V M A"

In [22]:
#Solution - From 5: which tagging has the highest probability?
step_5_highest = "<bos> N V A N"

By inspection, what is the accuracy of the best HMM labeling, given as a proportion of the words?

In [23]:
#TODO
example_maj_bigram_label_accuracy = "your answer here"

In [24]:
#Solution
example_maj_bigram_label_accuracy = 5/5

### Calculating the highest probability tagging - The Viterbi algorithm

Above, we merely asserted that the two highlighted paths are the two most probable, so that it was a simple matter to find the highest probability tagging by just comparing the probabilities of those two. But in general there can be a huge number of paths through a trellis such as this.

If there are $N$ tags and a sentence of length $M$, how many paths through the HMM trellis will there be (using big-O notation)?

In [25]:
#TODO
open_response_1 = "your answer here"

In [26]:
#Solution
open_response_1 = """
   At each of the M words there are N possibilities, 
   so the total is O(N^M), exponential in the sentence length.
   That's bad.
"""

The Viterbi algorithm, named after famed electrical engineer [Andrew Viterbi](https://en.wikipedia.org/wiki/Andrew_Viterbi), is an efficient dynamic programming algorithm for performing this (otherwise impractical) computation. We'll do the first few steps of the Viterbi algorithm for the example here.

Given a string of words $\vect{x} = \langle x_0, x_1, \ldots, x_M \rangle$ and a set of states (tags) $\vect{q} = \{q_0, q_1, \ldots, q_N\}$, the algorithm works by calculating a series of values $v_i(j)$ where $i$ ranges over the words in the sentence from $1$ to $M$ and $j$ ranges over the tags from $1$ to $N$. For simplicity, we'll assume an extra word and tag at the begining of the sentence, as above, so $x_0 = \texttt{<bos>}$ and $q_0 = \texttt{<bos>}$.
The definition for $v$ then is:

$$ 
\begin{align*}
  v_0(0) &= 1 \\
  v_0(j) &= 0  &\mbox{for $j > 0$} \\
  v_i(j) &= \max_{j'=1}^N v_{i-1}(j') \cdot a_{j' j} \cdot b_{j}(x_i)
         & \mbox{for $i > 0$}
\end{align*}
$$

For the sample sentence above (canners can canned fish), calculate (at least) the first two "layers" of the Viterbi algorithm, that is, $v_0$ and $v_1$, filling in the table below. (We've filled in the zero-th layer for you already, as per the first two lines in the definition of the Viterbi calculation.)

<!--TODO-->
|   | tag     | v_0  \<bos\> | v_1 canners | v_2 can | v_3 canned | v_4 fish |
|---|---------|--------------|-------------|---------|------------|----------|
| 0 | \<bos\> |    1         |             |         |            |          |
| 1 | N       |    0         |             |         |            |          |
| 2 | V       |    0         |             |         |            |          |
| 3 | M       |    0         |             |         |            |          |
| 4 | P       |    0         |             |         |            |          |
| 5 | A       |    0         |             |         |            |          |
| 6 | R       |    0         |             |         |            |          |

Doing this calculation by hand is painful, but it should make clear what's going on. At each point $v_i(j)$ in the table, we've calculated the probability of the best path through the trellis from the beginning of the sentence to the current word  $x_i$, starting in the start state and ending in the current state $q_j$. To get the maximum probability of all paths in the trellis for the full sentence ending in any state, we merely look up the maximum value of $v_M(j)$.

What is the complexity of filling in all of the entries in the Viterbi table? How does that compare with the complexity of the total number of paths through the trellis that you calculated above?

In [27]:
#TODO
open_response_2 = "your answer here"

In [28]:
#Solution
open_response_2 = """
    There are $O(NM)$ entries in the table, and each requires a calculation
    that takes a max over $N$ constant time calculations, so the total
    complexity is $O(N^2M)$. This is quadratic in $N$ rather than exponential, 
    and linear in $M$, far better than the brute force method of enumerating 
    all paths.
"""

---

To double-check your work, the cell below will rerun all of the autograder tests.

In [ ]:
grader.check_all()

## Submission

Make sure you have run all cells in your notebook in order before running the cell below, so that all images/graphs appear in the output. The cell below will generate a zip file for you to submit. **Please save before exporting!**

In [ ]:
# Save your notebook first, then run this cell to export your submission.
grader.export("lab2-3.ipynb")